<a href="https://colab.research.google.com/github/christina-rigsby/SpeedChallenge/blob/Similarity/Copy_of_Speed_Challenge_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Comma AI Speed Challenge**

  This notebook will contain (hopefully) all of the functions you need to import the data into your model.

  ***Be sure to train with GPU acceleration enabled***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Import Statements**

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from datetime import datetime
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D, Lambda, Dot


**Custom Data Generator**

This works (I think) for giving two sequential images to a Keras Functional model as well as the velocity associated with the second image.

At this point, don't worry about how this works. If you need something changed or fixed, just ask. This is the boring part anyways.

In [ ]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.direct = "./drive/My Drive/commai_dataset/"

    def __len__(self):
        'Denotes the number of batches per epoch'
        return len(self.list_IDs)

    def __getitem__(self, index):
        'Generate one batch of data'
        
        # Find list of IDs
        list_IDs_temp = self.list_IDs[index]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.load(self.direct+"data/data_" + list_IDs_temp +".npy")
        x1 = X[0:101,:,:,:]
        x2 = X[1:102,:,:,:]
        y = np.load(self.direct+"labels/label_" + list_IDs_temp +".npy")
        y = y[1:]

        return [x1, x2], y

**Define custom loss function**

This is not well tested, neither is it optimized. You might not even want to use this function.

Keras backend functions are a powerful tool for writing custom loss functions. To define a loss function it just has to accept *y_true* and *y_pred* as arguments and return a float.

To use your new loss function, change the argument in *model.compile()*.

In [ ]:
def sum_sq_err(y_true, y_pred):
    return tf.keras.backend.sum(tf.keras.backend.square(y_true - y_pred))

**Define the test-train split and create the Data Generator**

In [ ]:
params = {'dim': (110,320),
          'batch_size': 101,
          'n_classes': 1,
          'n_channels': 3,
          'shuffle': False}

train_data = []
train_label = []
valid_data = []
valid_label = []

for i in range(70):
    train_data.append("%03d" %i)
    train_data.append("%03d" %(i+100))

for i in range(70, 100):
    valid_data.append("%03d" %i)
    valid_data.append("%03d" %(i+100))

partition={'train':train_data, 'validation':valid_data}
labels = {'train': train_label,'validation':valid_label}

training_generator = DataGenerator(partition['train'], labels['train'], **params)
validation_generator = DataGenerator(partition['validation'], labels['validation'], **params)

**Define the input layers**

In [ ]:
input_A = tf.keras.layers.Input(shape=(110, 320, 3), name="first_image")
input_B = tf.keras.layers.Input(shape=(110, 320, 3), name="second_image")

In [ ]:
print(input_A)

Tensor("first_image:0", shape=(None, 110, 320, 3), dtype=float32)


**Define the model**

In [ ]:
#Sequential model won't work with two inputs
#model = keras.Sequential()

#model.add(Conv2D(64, (10,10), activation='relu'))
#model.add(MaxPooling2D())
#model.add(Conv2D(128, (7,7), activation='relu'))
#model.add(Flatten())
#model.add(Dense(1000, activation='sigmoid'))
#model = tf.keras.models.Model(inputs=[input_A, input_B], outputs=[out])

#Define model by specifying its forward pass:
def MyModel(inp):
  conv1 = Conv2D(64, (10,10), activation='relu')(inp)
  maxpool1 = MaxPooling2D()(conv1)
  conv2 = Conv2D(128, (7,7), activation = 'relu')(maxpool1)
  maxpool2 = MaxPooling2D()(conv2)
  conv3 = Conv2D(128, (4,4))(maxpool2)
  maxpool3 = MaxPooling2D()(conv3)
  conv4 = Conv2D(256, (4,4), activation='relu')(maxpool3)
  flat = Flatten()(conv4)
  out = Dense(4096, activation='sigmoid')(flat)
  return out 

Generate the feature vectors (encodings) for both images 

In [ ]:
encoding_A = MyModel(input_A)
encoding_B = MyModel(input_B) 

Add layer to compute absolute difference between encodings

In [ ]:
L1_layer = Lambda(lambda tensors:tf.keras.backend.abs(tensors[0]-tensors[1]))
L1_distance = L1_layer([encoding_A, encoding_B])

Add dense layer to generate the similarity score between images

In [ ]:
similarity = Dense(1, activation='sigmoid')(L1_distance)

In [ ]:
siamese_net = tf.keras.models.Model(inputs=[input_A, input_B], outputs=similarity)

**Model Details**

In [ ]:
siamese_net.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
first_image (InputLayer)        [(None, 110, 320, 3) 0                                            
__________________________________________________________________________________________________
second_image (InputLayer)       [(None, 110, 320, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 101, 311, 64) 19264       first_image[0][0]                
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 101, 311, 64) 19264       second_image[0][0]               
______________________________________________________________________________________________

**Declare the optimizer and loss function, then compile your *less ridiculous*  model**

In [ ]:
_optimizer = tf.keras.optimizers.Adam()
siamese_net.compile(optimizer=_optimizer, loss = sum_sq_err)

**Train using the fit_generator**

In [ ]:
siamese_net.fit_generator(generator=training_generator, validation_data=validation_generator, epochs=5)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/5
140/140 [==============================] - 416s 3s/step - loss: 21411.1719 - val_loss: 14864.7969
Epoch 2/5
140/140 [==============================] - 435s 3s/step - loss: 21388.8359 - val_loss: 14864.7969
Epoch 3/5
140/140 [==============================] - 431s 3s/step - loss: 21388.8320 - val_loss: 14864.7969
Epoch 4/5
140/140 [==============================] - 431s 3s/step - loss: 21388.8281 - val_loss: 14864.7969
Epoch 5/5
140/140 [==============================] - 436s 3s/step - loss: 21388.8320 - val_loss: 14864.7969


In [ ]:
!ls

drive  sample_data
